In [ ]:
""" Just a work bench"""
# %load_ext tensorboard

import os
import json
from typing import List
from pprint import pprint
from datetime import datetime

from dotenv import load_dotenv

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, AutoTokenizer, AutoConfig, TFDistilBertModel, TFBertModel, TFTrainingArguments
import tensorflow as tf
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_val_predict, cross_validate, KFold
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score, explained_variance_score

from tc_data import TopCoder
from run_classification import build_dataset
from model_tcpm_distilbert import (
    TCPMDistilBertClassification,
    build_tcpm_model_distilbert_classification,
    build_tcpm_model_distilbert_regression
)

load_dotenv()
pd.set_option('display.max_rows', 800)


In [ ]:
tc = TopCoder()

In [ ]:
cha_info = tc.get_filtered_challenge_info()

In [ ]:
with sns.axes_style('darkgrid'):
    fig, axes = plt.subplots(2, 2, figsize=(8, 8), dpi=200)
    axes = [*axes[0], *axes[1]]
    for i, col in enumerate(('max_score', 'min_score', 'avg_score', 'std_score')):
        x_plot = np.linspace(0, 2500, 2500) # plot the polynominal regression line

        reg = LinearRegression()
        reg.fit(X=cha_info['total_prize'].to_numpy().reshape(-1, 1), y=cha_info[col].to_numpy())
        y_line = reg.predict(x_plot.reshape(-1, 1))
        
        poly_reg = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
        poly_reg.fit(X=cha_info['total_prize'].to_numpy().reshape(-1, 1), y=cha_info[col].to_numpy())
        y_curve = poly_reg.predict(x_plot.reshape(-1, 1))
        
        ax = axes[i]
        sns.scatterplot(
            data=cha_info,
            x='total_prize',
            y=col,
            s=8,
            alpha=0.5,
            linewidth=0,
            ax=ax
        )
        sns.lineplot(
            x=x_plot,
            y=y_line,
            linewidth=1.5,
            alpha=0.75,
            color='orange',
            ax=ax,
        )
        sns.lineplot(
            x=x_plot,
            y=y_curve,
            linewidth=1.5,
            alpha=0.75,
            color='red',
            ax=ax
        )

        ax.set_title(' '.join([w.capitalize() for w in col.split('_')]))
        
    fig.tight_layout()

In [ ]:
with sns.axes_style('darkgrid'):
    fig, axes = plt.subplots(1, 2, figsize=(8, 4), dpi=200)
    x_plot = np.linspace(0, 2500, 2500)
    
    for i, col in enumerate(('number_of_registration', 'sub_reg_ratio')):
        reg = LinearRegression()
        reg.fit(X=cha_info['total_prize'].to_numpy().reshape(-1, 1), y=cha_info[col].to_numpy())
        y_line = reg.predict(x_plot.reshape(-1, 1))
        
        poly_reg = make_pipeline(PolynomialFeatures(degree=3), LinearRegression())
        poly_reg.fit(X=cha_info['total_prize'].to_numpy().reshape(-1, 1), y=cha_info[col].to_numpy())
        y_curve = poly_reg.predict(x_plot.reshape(-1, 1))
        
        ax = axes[i]
        sns.scatterplot(
            data=cha_info,
            x='total_prize',
            y=col,
            s=8,
            alpha=0.5,
            linewidth=0,
            ax=ax
        )
        sns.lineplot(
            x=x_plot,
            y=y_line,
            linewidth=1.5,
            alpha=0.75,
            color='orange',
            ax=ax,
        )
        sns.lineplot(
            x=x_plot,
            y=y_curve,
            linewidth=1.5,
            alpha=0.75,
            color='red',
            ax=ax
        )

        ax.set_title(' '.join([w.capitalize() for w in col.split('_')]))
    
    
    fig.tight_layout()

In [ ]:
cha_info.columns

In [ ]:
X = cha_info.reindex([
    'number_of_platforms',
    'number_of_technologies',
    'project_id',
    'challenge_duration',
    'total_prize'
], axis=1).join(cha_info['subtrack_category'].cat.codes.rename('subtrack_category'))

In [ ]:
y = cha_info['avg_score']

In [ ]:
def mre(y_true, y_pred, sample_weight=None):
    return np.mean(np.abs(y_true - y_pred) / y_true)

In [ ]:
reg = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
scores = cross_validate(
    reg, 
    X.to_numpy(), 
    y.to_numpy(), 
    cv=10, 
    scoring={
        'mre': make_scorer(mre, greater_is_better=False),
        'mae': make_scorer(mean_absolute_error, greater_is_better=False),
        'mse': make_scorer(mean_squared_error, greater_is_better=False),
        'r2': make_scorer(r2_score),
        'explained_var': make_scorer(explained_variance_score)
    }
)

In [ ]:
pd.DataFrame(scores).mean().abs()

In [ ]:
y.value_counts()